# VGG16 finetuned with image augmentation
## author: Jiajun Dai
### First model: VGG16

In [ ]:
# connect with google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install opencv-python

Convertor to resize images to 48 x 48 (only run this once)

In [ ]:
# from PIL import Image
# import os
# import cv2

# # List files in the folder
# files = os.listdir(data_folder_path)
# print(files)

# # Create a new folder to save the results
# save_path = 'content/drive/MyDrive/data/processed_data'
# os.makedirs(save_path, exist_ok=True)

# for emo_folder in files:
#     emo_folder_path = os.path.join(data_folder_path, emo_folder)
#     for img_file in os.listdir(emo_folder_path):
#       img_path = os.path.join(emo_folder_path, img_file)
#       # Read the image
#       img = cv2.imread(img_path)
#       # Resize the image to 48 x 48 pixels
#       img_resized = cv2.resize(img, (48, 48))
#       # Convert the image to grayscale
#       img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
#       # Save the result
#       output_path = os.path.join(emo_folder_path, img_file)
#       cv2.imwrite(output_path, img_gray)
#       print(f"created processed_{img_file} in {output_path}")

In [ ]:
# !pip install tensorflow

In [ ]:
import os

# Set the paths
input_data_path = '/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/Datasets/train_data_processed'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
image_generator = ImageDataGenerator(
    # This process is also called Normalizing the input.
    # Scaling every images to the same range [0,1] will make images contributes more evenly to the total loss
    rescale = 1./255,
    # set validation set scale
    validation_split = 0.2,
    rotation_range=5,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # fill_mode='nearest'
    )

In [ ]:
train_dataset  = image_generator.flow_from_directory(directory = input_data_path,
                                                  target_size = (48,48),
                                                  color_mode='rgb',
                                                  class_mode = 'categorical',
                                                  subset = 'training',
                                                  batch_size = 64,
                                                  shuffle=True)

Found 24954 images belonging to 7 classes.


In [ ]:
print(train_dataset.image_shape)
# Access the next batch of data
x_batch, y_batch = train_dataset.next()
print(x_batch.shape)
print(y_batch.shape)

(48, 48, 3)
(64, 48, 48, 3)
(64, 7)


In [ ]:
validation_dataset = image_generator.flow_from_directory(directory = input_data_path,
                                                  target_size = (48,48),
                                                  color_mode='rgb',
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 64)

Found 6235 images belonging to 7 classes.


Retriving pretrained VGG16 model from tensorflows keras

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(48,48,3),include_top=False,weights="imagenet")

In [ ]:
# Freezing Layers

for layer in base_model.layers[:-4]:
    layer.trainable=False

In [ ]:
# Building Model

model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

In [ ]:
# Model Summary

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 dropout_3 (Dropout)         (None, 1, 1, 512)         0         
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 batch_normalization_4 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_11 (Dense)            (None, 32)                16416     
                                                                 
 batch_normalization_5 (Bat  (None, 32)                128       
 chNormalization)                                     

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint

# taken from old keras source code
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Set up LearningRateScheduler
lr_scheduler = LearningRateScheduler(scheduler, verbose=1)
# lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 20,verbose = 1,factor = 0.50, min_lr = 1e-10)
checkpoint_filepath = '/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/VGG16/VGG16_weights_epoch_{epoch:02d}_val_loss_{val_loss:.4f}.h5'
# Set up ModelCheckpoint
checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_loss',            # Monitor the validation loss
    verbose=1,                     # Logging level
    save_best_only=True,           # Save only the best model
    save_weights_only=False,       # If True, only weights are saved
    mode='auto',                   # Auto mode means the direction is inferred
    save_freq='epoch'              # Save after each epoch
)

# Set up EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
)

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      f1_score,
]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=METRICS)

In [ ]:
history=model.fit(train_dataset,validation_data=validation_dataset,epochs = 20,batch_size=64,callbacks=[lr_scheduler, early_stopping, checkpoint])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/20
 62/390 [===>..........................] - ETA: 38s - loss: 2.0973 - accuracy: 0.8506 - precision: 0.2058 - recall: 0.0161 - auc: 0.5481 - f1_score: 0.0295

Evaluate trained VGG16 model

In [ ]:
!pip install mtcnn
import os
import cv2
import numpy as np
from mtcnn import MTCNN

data_path = "/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/Datasets/test"
INPUT_SIZE = (48, 48)
classes = ["surprise", "sad", "neutral", "happy", "fear", "disgust", "angry"]
detector = MTCNN()

face_images = []
actual_labels = []

for emotion_folder in os.listdir(data_path):
    emotion_path = os.path.join(data_path, emotion_folder)
    if os.path.isdir(emotion_path):
       # counter = 0  # Initialize a counter for each folder
        for img_file in os.listdir(emotion_path):
           # if counter < 5:  # Process only 5 images per folder
                img_path = os.path.join(emotion_path, img_file)
                image = cv2.imread(img_path)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                faces = detector.detect_faces(image_rgb)

                for face in faces:
                    x, y, width, height = face['box']
                    face_image = image_rgb[y:y+height, x:x+width]
                    resized_face = cv2.resize(face_image, INPUT_SIZE)
                    face_images.append(resized_face)
                    actual_labels.append(emotion_folder)  # Folder name as the actual label

                #counter += 1  # Increment the counter
          #  else:
               # break  # Move to the next folder

# Convert lists to numpy arrays
face_images = np.array(face_images)

In [ ]:
import matplotlib.pyplot as plt

def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

print(acc)
print(val_acc)
print(loss)
print(val_loss)

smooth_acc = smooth_curve(acc)
smooth_val_acc = smooth_curve(val_acc)
smooth_loss = smooth_curve(loss)
smooth_val_loss = smooth_curve(val_loss)

epochs = range(1, len(acc) + 1)

# Plot training & validation accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, smooth_acc, 'b', label='Smoothed Training accuracy')
plt.plot(epochs, smooth_val_acc, 'r', label='Smoothed Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, smooth_loss, 'b', label='Smoothed Training loss')
plt.plot(epochs, smooth_val_loss, 'r', label='Smoothed Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.save("/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/model_data/VGG16ft.h5")

In [ ]:
custom_objects = {'f1_score': f1_score}
load_model_vgg16ft=tf.keras.models.load_model("/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/model_data/VGG16ft.h5", custom_objects=custom_objects)

In [ ]:
classes = ["surprise", "sad", "neutral", "happy", "fear", "disgust", "angry"]

In [ ]:
predictions = load_model_vgg16ft.predict(face_images)
predictions = predictions.reshape(predictions.shape[0], -1)
print(face_images[0].shape)
print(predictions.shape)
print(np.argmax(predictions[0]))
# print(classes[np.argmax(predictions[2])])
# Convert predictions to labels
predicted_labels = [classes[np.argmax(pred)] for pred in predictions]

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Convert actual string labels to indices
actual_indices = [classes.index(label) for label in actual_labels]

# Convert predicted string labels to indices
predicted_indices = [classes.index(label) for label in predicted_labels]

# Calculate accuracy
accuracy = accuracy_score(actual_indices, predicted_indices)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Generate confusion matrix
cm = confusion_matrix(actual_indices, predicted_indices)

# Generate and print classification report
report = classification_report(actual_indices, predicted_indices, target_names=classes)
print(report)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title(f'Confusion Matrix, Accuracy: {accuracy * 100:.2f}%')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


## VGG16 without finetune

In [ ]:
import os
from math import ceil
import cv2

In [ ]:
datadirectory="/content/drive/Shareddrives/CMPE 258 - Deep Learning Project/Software Programs/Datasets/train_data_processed"

In [ ]:
classes=["surprise","sad","neutral","happy","fear","disgust","angry"]
img_size=48

In [ ]:
from logging import exception
training_data=[]
def create_training_data():
  for category in classes:
    path=os.path.join(datadirectory, category)
    class_num=classes.index(category)
    for img in os.listdir(path):
      try:
        img_array=cv2.imread(os.path.join(path,img))
        new_array=cv2.resize(img_array,(img_size,img_size))
        training_data.append([new_array,class_num])
      except exception as e:
        pass
  return training_data

In [ ]:
train_data = create_training_data()

In [ ]:
import random
random.shuffle(train_data)

In [ ]:
x=[]
y=[]
for features,label in train_data:
  x.append(features)
  y.append(label)
# Convert list x to numpy array
x=np.array(x).reshape(-1,img_size,img_size,3)

In [ ]:
x.shape

In [ ]:
# Convert list y to numpy array
y=np.array(y)

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (80% training, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y)

In [ ]:
print("Training Data:")
print("Features (x_train):", x_train.shape)
print("Labels (y_train):", y_train.shape)

print("\nValidation Data:")
print("Features (x_val):", x_val.shape)
print("Labels (y_val):", y_val.shape)

In [ ]:
base_model_2 = tf.keras.applications.VGG16(input_shape=(48,48,3),include_top=False,weights="imagenet")

In [ ]:
# Freezing Layers

for layer in base_model_2.layers[:-4]:
    layer.trainable=False

In [ ]:
# Building Model

model=Sequential()
model.add(base_model_2)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
EPOCHS = 20
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    batch_size=32,
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)

In [ ]:
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

print(acc)
print(val_acc)
print(loss)
print(val_loss)

smooth_acc = smooth_curve(acc)
smooth_val_acc = smooth_curve(val_acc)
smooth_loss = smooth_curve(loss)
smooth_val_loss = smooth_curve(val_loss)

epochs = range(1, len(acc) + 1)

# Plot training & validation accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, smooth_acc, 'b', label='Smoothed Training accuracy')
plt.plot(epochs, smooth_val_acc, 'r', label='Smoothed Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, smooth_loss, 'b', label='Smoothed Training loss')
plt.plot(epochs, smooth_val_loss, 'r', label='Smoothed Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(face_images)

# Convert predictions to labels
predicted_labels = [classes[np.argmax(pred)] for pred in predictions]

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Convert actual string labels to indices
actual_indices = [classes.index(label) for label in actual_labels]

# Convert predicted string labels to indices
predicted_indices = [classes.index(label) for label in predicted_labels]

# Calculate accuracy
accuracy = accuracy_score(actual_indices, predicted_indices)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Generate confusion matrix
cm = confusion_matrix(actual_indices, predicted_indices)

# Generate and print classification report
report = classification_report(actual_indices, predicted_indices, target_names=classes)
print(report)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()